## Intersecting and Filtering mapped reads

__running time__: < 10 min

To produce the main Hi-C interaction matrix we need to join the two parsed tsv files and keep only pairs of reads that are both mapped.

After the intersection of the parsed files the read-ends are filtered in order to keep only informative (valid) pairs.

The filters available in TADbit are:
1. __Self-circle__: both read-ends are mapped to the same RE fragment in *opposed* orientation.
2. __Dangling-end__: both read-ends are mapped to the same RE fragment in *facing* orientation.
3. __Error__: both read-ends are mapped to the same RE fragment in the same orientation.
4. __Extra dangling-end__: the read-ends are mapped to different RE fragments in *facing* orientation, but are close enough (< *max_molecule_length* bp) from the RE cut-site to be considered part of adjacent RE fragments that were not separated by digestion. The *max_molecule_length* parameter can be inferred from the *fragment_size* function previously detailed.
5. __Too close from RE sites (or semi-dangling-end)__: the start position of one of the read-end is too close (5 bp by default) from the RE cutting site. 
6. __Too short__: one of the read-ends is mapped to RE fragments of less than 75bp. These are removed since there is ambiguity on where the read-end is mapped as it could also belong to any of the two neighboring RE fragments.
7. __Too large__: the read-ends are mapped to long RE fragments (default: 100 kb, P < 10-5 to occur in a randomized genome) and they likely represent poorly assembled or repetitive regions.
8. __Over-represented__: the read-ends coming from the top 0.5% most frequently detected RE fragments, they may represent PCR artefacts, random breaks, or genome assembly errors. 
9. __PCR artefacts or duplicated__: the combination of the start positions, mapped length, and strands of both read-ends are identical. In this case, only one copy is kept.
10. __Random breaks__: the start position of one read-end is too far (> *minimum_distance_to_RE*) from the RE cut-site. These are produced most probably by non-canonical enzyme activity or by random physical breakage of the chromatin. Note, that to filter all these types of fragments the minimum_distance_to_RE parameter should be larger than the *maximum_fragment_length*.

![Filters](images/Filters.jpeg)

With TADbit tools the intersection and filtering steps are conducted with the `tadbit filter` tool. We will use it with the default settings:

In [1]:
%%bash

tadbit filter -w ../results/PSC_rep1

Getting intersection between read 1 and read 2
Get insert size...
  - median insert size = 144.0
  - median absolution of insert size = 50.0
  - max insert size (when a gap in continuity of > 10 bp is found in fragment lengths) = 512
   Using the maximum continuous fragment size(512 bp) to check for pseudo-dangling ends
   Using maximum continuous fragment size plus the MAD (562 bp) to check for random breaks
identify pairs to filter...
Filtered reads (and percentage of total):

                   Mapped both  :   11,070,407 (100.00%)
  -----------------------------------------------------
   1-               self-circle :       16,432 (  0.15%)
   2-              dangling-end :    1,020,603 (  9.22%)
   3-                     error :      171,074 (  1.55%)
   4-        extra dangling-end :    2,150,926 ( 19.43%)
   5-        too close from RES :    2,239,071 ( 20.23%)
   6-                 too short :      333,421 (  3.01%)
   7-                 too large :          103 (  0.00%)
   8

[bam_sort_core] merging from 0 files and 32 in-memory blocks...


### Inspect results

We can list the generated files in the working directory in the subfolder `03_intersection`:

In [2]:
%%bash

ls ../results/PSC_rep1/03_filtered_reads/

all_r1-r2_intersection_f69652928b.tsv
all_r1-r2_intersection_f69652928b.tsv_dangling-end.tsv
all_r1-r2_intersection_f69652928b.tsv_duplicated.tsv
all_r1-r2_intersection_f69652928b.tsv_error.tsv
all_r1-r2_intersection_f69652928b.tsv_extra_dangling-end.tsv
all_r1-r2_intersection_f69652928b.tsv_over-represented.tsv
all_r1-r2_intersection_f69652928b.tsv_random_breaks.tsv
all_r1-r2_intersection_f69652928b.tsv_self-circle.tsv
all_r1-r2_intersection_f69652928b.tsv_too_close_from_RES.tsv
all_r1-r2_intersection_f69652928b.tsv_too_large.tsv
all_r1-r2_intersection_f69652928b.tsv_too_short.tsv
intersection_f69652928b.bam
intersection_f69652928b.bam.bai
valid_r1-r2_intersection_f69652928b.tsv


The main intersection file is the `all_r1-r2_intersection_<id>.tsv` file. We can check its structure:

In [4]:
%%bash

head ../results/PSC_rep1/03_filtered_reads/all_r1-r2_intersection_f69652928b.tsv -n 3

# CRM chr3	159745316
SRR5344969.sra.233045900	chr3	3000237	0	75	1	3005125	chr3	44068461	1	75	44067902	44068553
SRR5344969.sra.109231286~2~	chr3	3001338	0	28	1	3005125	chr3	101946800	1	301	101946316	101947097


Intersection files are tsv files containing information of mapped read pairs. They have the following columns:
1. Extended sequence identifier as in the parsed file.
2. Chromosome of the first pair.
3. Start position of the first pair.
4. Strand of the first pair.
5. Length of the sequence of the first pair.
6. Position of the left flanking restriction enzyme site of the first pair.
7. Position of the right flanking restriction enzyme site of the first pair.
8. Chromosome of the mate pair.
9. Start position of the mate pair.
10. Strand of the mate pair.
11. Length of the sequence of the mate pair.
12. Position of the left flanking restriction enzyme site of the mate pair.
13. Position of the right flanking restriction enzyme site of the mate pair

The other `all_r1-r2_intersection_<id>.tsv_` files contain the sequence identifiers of the read-pairs belonging to the different filters.
For example the sequence identifiers of the dangling-ends are in the `all_r1-r2_intersection_<id>.tsv_dangling-end.tsv`:

In [5]:
%%bash

head ../results/PSC_rep1/03_filtered_reads/all_r1-r2_intersection_f69652928b.tsv_dangling-end.tsv -n 3

SRR5344969.sra.171381778
SRR5344969.sra.48401438
SRR5344969.sra.203935105~2~


The intersection file `valid_r1-r2_intersection_<id>.tsv` contains the read-pairs that passed the filter. By default only filters 1,2,3,4,6,7,9,10 from the list are applied.

The main output of the `tadbit filter` tool is the `intersection_<id>.bam`. The Binary Alignment/Map (BAM) is a compressed binary format to store a Sequence Alignment/Map (SAM) file.  
TADbit uses the BAM format to store the information included in the intersection file including the filter classification of read pairs. The advantages of this binary format are multiple: the reduced size of the files, the fast access to sections of the data thanks to the indexation, the standard access to the information through samtools, etc.

__Note__: The fields we use in TADbit to generate a BAM file are not the conventional ones, we modify them as follows to store only the necessary information for the remaining part of the analysis.

Using samtools we can inspect the information contained in the BAM file:

In [6]:
%%bash

samtools view ../results/PSC_rep1/03_filtered_reads/intersection_f69652928b.bam | head -n 3

SRR5344969.sra.233045900	576	chr3	3000237	0	75P	=	44068461	75	*	*	TC:i:1	S1:i:0	S2:i:1
SRR5344969.sra.109231286~2~	576	chr3	3001338	0	28P	=	101946800	301	*	*	TC:i:1	S1:i:0	S2:i:1
SRR5344969.sra.25771537~2~	576	chr3	3001338	0	29P	=	12424423	117	*	*	TC:i:1	S1:i:0	S2:i:1


This is a description of the information contained in the columns of a TADbit BAM file:
1. Extended sequence identifier as in the parsed file.
2. Filtering flag (binary mask for the application of the 10 filters previously described):

   1 self-circle  
   2 dangling-end  
   3 error  
   4 extra dangling-end  
   5 too close from RES  
   6 too short  
   7 too large  
   8 over-represented  
   9 duplicated  
   10 random breaks  
   11 inter-chromosomal  
   
   For example if we want to keep only pairs of read-ends that are excelusively inter-fragment contacts and that are not duplicated, we would apply filters 1, 2, 3 (self-circle, dangling-ends, errors) and 9 (duplicated) resulting in a binary number like this: 00100000111 which translates in decimal: 263. We could thus obtain these read-pairs with `samtools view -F 263`.
3. A read pair may belong to several categories.
4. Chromosome ID of the first pair.
5. Position of the first pair.
6. MAPQ is set to 0.
7. Pseudo CIGAR replaced by the mapped length of the first read-end, and information about current copy (each pair is present twice in the BAM, P: first copy, S: second copy)
8. Chromosome ID of the second pair.
9. Position of the second pair.
10. Mapped length of the second pair.
11. Sequence is missing (*)
12. Quality is missing (*)
13. Tags:  
    TC tag indicating single (1) or multi contact (the number of times a given sequenced fragment is involved in a pairwise contact).  
    S1 and S2 tags are the strand orientation of the first and mate reads.


### Check applied filters

`tadbit filter` estimates the parameters used in the filters from the histogram of the dangling-ends lenghts. The plot is saved as `histogram_fragment_sizes_<id>.pdf` in the working directory:

In [1]:
from IPython.display import IFrame

IFrame("../results/PSC_rep1/histogram_fragment_sizes_f69652928b.pdf", width=800, height=600)


The `max_molecule_length` parameter used to filter-out pseudo-dangling-ends is  extracted from the histogram. It is estimated as the maximum continuous fragment size: when a gap in continuity of > 10 bp is found in fragment lengths.  
It is calculated as the first window where 10 consecutive sizes are counted less than a given value (the sum of all sizes divided by 100000); in our case is set to 512 bp.

The `min_distance_to_re`, that affects the detection of random breaks, should be large enough in order to contain almost all the fragments. It is estimated as the maximum continuous fragment size plus the Median Absolute Deviation (more robust than the standard deviation for non-normal distributions https://en.wikipedia.org/wiki/Median_absolute_deviation); in our case is set to 562 bp


The output of `tadbit tool` reports the number of valid and filtered read-pairs:

In [8]:
%%bash

tadbit describe -w ../results/PSC_rep1/ -t 7

,----------------.
| FILTER_OUTPUTs |
,----.--------.--------------------.-----------.---------.-------.
| Id | PATHid |               Name |     Count | Applied | JOBid |
|----+--------+--------------------+-----------+---------+-------|
|  1 |     20 |        self-circle |    16,432 |    True |     5 |
|  2 |     21 |       dangling-end | 1,020,603 |    True |     5 |
|  3 |     22 |              error |   171,074 |    True |     5 |
|  4 |     23 | extra dangling-end | 2,150,926 |    True |     5 |
|  5 |     24 | too close from RES | 2,239,071 |   False |     5 |
|  6 |     25 |          too short |   333,421 |    True |     5 |
|  7 |     26 |          too large |       103 |    True |     5 |
|  8 |     27 |   over-represented |    70,463 |   False |     5 |
|  9 |     28 |         duplicated |   218,902 |    True |     5 |
| 10 |     29 |      random breaks |    22,850 |    True |     5 |
| 11 |     15 |        valid-pairs | 7,427,477 |         |     5 |
'----^--------^---------

### Plot the interaction matrix

To generated a binned matrix from the interaction BAM file we use `tadbit bin`. In this case we will plot the matrix of chromosome 3 at 50kb resolution:

In [ ]:
%%bash

tadbit bin -w ../results/PSC_rep1/ --only_plot  \
    -c chr3 --resolution 50000 \
    --cmap Reds --format png

In [ ]:
%%bash

ls ../results/PSC_rep1/05_sub-matrices/

The generated matrix image is stored in the working directory under the `05_sub-matrices` subfolder.


In [ ]:
from IPython.display import Image
Image(filename='../results/PSC_rep1/05_sub-matrices/raw_chr3_50kb_f12ce89c57.png') 

In [4]:
%%bash

tadbit clean -w ../results/PSC_rep1/ -j 12 --delete

deleting 1 files
  x ../results/PSC_rep1/05_sub-matrices/nrm_chr3:3395-3545_10kb_c8e49087d1.png
cleaning table: PATHs
 - deleted 1 elements with JOBid = 12
cleaning table: JOBs
 - deleted 1 elements with Id = 12
cleaning table: MAPPED_INPUTs
cleaning table: MAPPED_OUTPUTs
cleaning table: PARSED_OUTPUTs
cleaning table: INTERSECTION_OUTPUTs
cleaning table: FILTER_OUTPUTs
cleaning table: NORMALIZE_OUTPUTs
cleaning table: SEGMENT_OUTPUTs


In [3]:
%%bash

tadbit describe -w ../results/PSC_rep1/ -t 2

,------.
| JOBs |
,----.-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------.---------------------.---------------------.-----------.----------------.
| Id |                                                                                                                                                                                                                                                                                      Parameters |         Launch_time |         Finish_time |      Type | Parameters_md5 |
|----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- True or False? In case it is false try to correct the sentence:
 1. Valid-pairs are always more than 50% in any Hi-C experiment.
 2. The filtering step is meant to remove read-pairs generated from experimental errors.
 3. In both self-circles and dangling-ends the paired-reads are mapped on the same RE fragment.
 4. All the filters must be applied in all the analysis.
- Could you explain the event of a Random-break and how one can spot it from analysing the mapped read-pair?
- Could you explain the origin of a duplicated read during the Hi-C experiment?